<a href="https://colab.research.google.com/github/Anastasia-Lyashuk/Portfolio/blob/main/yandex_music_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Обзор данных




### 1.1.Импорт библиотеки pandas

In [ ]:
import pandas as pd # импорт библиотеки pandas

### 1.2. Загрузим данные из CSV-файлов в переменную df

In [ ]:
df = pd.read_csv('/datasets/yandex_music_project.csv') # чтение файла с данными и сохранение в df

### 1.3. Выведем на экран первые десять строк таблицы

In [ ]:
print(df.head(10))# получение первых 10 строк таблицы df

### 1.4. Изучим данные в датафрейме

In [ ]:
df.info() # получение общей информации о данных в таблице df

## 2. Предобработка данных


### 2.1. Стиль заголовков


In [ ]:
print(df.columns)# перечень названий столбцов таблицы df

In [ ]:
df = df.rename(columns={'  userID':'user_id','Track':'track','  City  ':'city','Day':'day'}) # переименование столбцов

In [ ]:
print(df.columns) # проверка результатов - перечень названий столбцов

### 2.2. Пропуски значений


In [ ]:
print(df.isna().sum()) # подсчёт пропусков

In [ ]:
columns_to_replace = df[['track', 'artist', 'genre']]
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'

In [ ]:
print(df.isna().sum()) # подсчёт пропусков

### 2.3. Дубликаты


In [ ]:
print(df.duplicated().sum()) # подсчёт явных дубликатов

In [ ]:
df = df.drop_duplicates() # удаление явных дубликатов

In [ ]:
print(df.duplicated().sum()) # проверка на отсутствие дубликатов

In [ ]:
genre_sort = df.sort_values('genre')

print(genre_sort['genre'].unique())

# Просмотр уникальных названий жанров

In [ ]:
duplicates = ['hip','hop','hip-hop']
name = 'hiphop'
df['genre'] = df['genre'].replace(duplicates,name)
    # Устранение неявных дубликатов

In [ ]:
genre_sort = df.sort_values('genre')

print(genre_sort['genre'].unique())
# Проверка на неявные дубликаты

## 3. Проверка гипотез

### 3.1. Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.


In [ ]:
track_grouping = df.groupby('city')['time']
track_grouping = track_grouping.count()
print(track_grouping)
# Подсчёт прослушиваний в каждом городе

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.



In [ ]:
track_grouping_day = df.groupby('day')['time']
track_grouping_day = track_grouping_day.count()
print(track_grouping_day)
# Подсчёт прослушиваний в каждый из трёх дней

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [ ]:
def number_tracks(day, city):
    track_list = df[df['day'] == day]
    track_list = track_list[track_list['city'] == city]
    track_list_count = track_list['user_id'].count()
    return track_list_count



# <создание функции number_tracks()>
# Функция для подсчёта прослушиваний для конкретного города и дня.

In [ ]:
print(number_tracks('Monday','Moscow')) # количество прослушиваний в Москве по понедельникам

In [ ]:
print(number_tracks('Monday', 'Saint-Petersburg'))# количество прослушиваний в Санкт-Петербурге по понедельникам

In [ ]:
print(number_tracks('Wednesday','Moscow')) # количество прослушиваний в Москве по средам

In [ ]:
print(number_tracks('Wednesday','Saint-Petersburg'))# количество прослушиваний в Санкт-Петербурге по средам

In [ ]:
print(number_tracks('Friday','Moscow'))# количество прослушиваний в Москве по пятницам

In [ ]:
print(number_tracks('Friday','Saint-Petersburg')) # количество прослушиваний в Санкт-Петербурге по пятницам

In [ ]:
data = [['Москва', 15740, 11056, 15945],['Санкт-Петербург', 5614, 7003,5895]]
columns = ['city','monday','wednesday','friday']
info = pd.DataFrame(data=data, columns=columns)
print(info)
# Таблица с результатами

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### 3.2. Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [ ]:
moscow_general = df[df['city'] == 'Moscow']
# получение таблицы moscow_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Moscow'


In [ ]:
spb_general = df[df['city']=='Saint-Petersburg']
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'


In [ ]:
def genre_weekday(df, day, time1, time2):
    # последовательная фильтрация
    # оставляем в genre_df только те строки df, у которых день равен day
    genre_df = df[df['day'] == day]
    # ваш код здесь
    # оставляем в genre_df только те строки genre_df, у которых время меньше time2
    genre_df = genre_df[genre_df['time'] < time2]
    # ваш код здесь
    # оставляем в genre_df только те строки genre_df, у которых время больше time1
    genre_df = genre_df[genre_df['time'] > time1]  # ваш код здесь
    # сгруппируем отфильтрованный датафрейм по столбцу с названиями жанров, возьмём столбец genre и посчитаем кол-во строк для каждого жанра методом count()
    genre_df_grouped = genre_df.groupby('genre')['genre'].count() # ваш код здесь
    # отсортируем результат по убыванию (чтобы в начале Series оказались самые популярные жанры)
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    # ваш код здесь
    # вернём Series с 10 самыми популярными жанрами в указанный отрезок времени заданного дня
    return genre_df_sorted[:10]



In [ ]:
print(genre_weekday(moscow_general, 'Monday', '07:00', '11:00')) # вызов функции для утра понедельника в Москве


In [ ]:
print(genre_weekday(spb_general, 'Monday', '07:00', '11:00')) # вызов функции для утра понедельника в Петербурге

In [ ]:
print(genre_weekday(moscow_general, 'Friday', '17:00', '23:00')) # вызов функции для вечера пятницы в Москве

In [ ]:
print(genre_weekday(spb_general, 'Friday', '17:00', '23:00')) # вызов функции для вечера пятницы в Петербурге

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### 3.3. Жанровые предпочтения в Москве и Петербурге



Гипотеза: В Петербурге чаще всего слушают музыку жанра рэп.  А Москва преобладает поп-музыка.

In [ ]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)


In [ ]:
print(moscow_genres.head(10)) # просмотр первых 10 строк moscow_genres

In [ ]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)


In [ ]:
print(spb_genres.head(10))
# просмотр первых 10 строк spb_genres

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


## 4. Выводы

После проверки трех гипотез можно предположить следующее:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге. Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.
